In [1]:
%config IPCompleter.greedy=True
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))

In [3]:
# Import libraries and define constants

import os
from datetime import datetime
from free_bwacs_model import FreeBWACS
from src.helpers import sorted_alphanumeric, save_results_on_file

RUNS = 10
SAVE_RESULTS = True
SAVE_IMGS = False
RESULTS_FILE_WITH_HEADER = True
BASE_RESULTS_FOLDER = 'results'


In [4]:
# Execution of runs for instances files

instances_with_errors = []
instances_folder = 'instances/CVRPLIB/CMT'
instances = sorted_alphanumeric([instance_name.replace(
    '.vrp', '') for instance_name in os.listdir(instances_folder)])
cluster_type = 'kmeans' # 'kmeans' or 'kmedoids'
ant_type = 'fa' # 'fa', 'ra' or 'random'

print(instances)

# Validation of folder to save results
if SAVE_RESULTS:
    if not os.path.exists(BASE_RESULTS_FOLDER):
        os.makedirs(BASE_RESULTS_FOLDER)
    
# Start running the algorithm
for instance in instances:
    now = datetime.now()
    solution_list = []

    for run in range(RUNS):
        file_name = f'{str(instance).lower()}_{ant_type}_{cluster_type}_run{run}'
        results_folder = f'{instance}/{now.strftime("%Y%m%d_%H%M%S")}'

        if SAVE_RESULTS:
            os.makedirs(f'{BASE_RESULTS_FOLDER}/{instance}', exist_ok=True)
            save_results_on_file(f'{BASE_RESULTS_FOLDER}/{instance}', f'{file_name}.txt', RESULTS_FILE_WITH_HEADER, solution_list, instance, cluster_type)

        if SAVE_IMGS:
            os.makedirs(
                f'{BASE_RESULTS_FOLDER}/{instance}/imgs', exist_ok=True)

        try:
            bwacs = FreeBWACS(
                instance=f'{instances_folder}/{instance}',
                max_nodes=9999,
                cluster_type=cluster_type,
                metric='euclidian',
                tare_percentage=0.15,
                max_iterations=100,
                total_ant_divider=1,
                only_k_optimum=True,
                start_ant_on_best_nodes=True,
                # heuristic_type will let you choice what information gotta build the heuristic information matrix
                # 0 - Only distances data / Best beta: 5
                # 1 - Only energies data / Best gamma: 5
                # 2 - Distances data * energies data / Best parameters: 5 - 5
                # 3 - Only distances data of Savings algorithm / Best delta: 1
                # 4 - Only energies data of Savings algorithm / Best eta: 2
                # 5 - Distances data * distances savings data / Best parameters: 4 - 5
                # 6 - Energies data * energies savings data / Best parameters: 5 - 5
                # 7 - Distances data * energies data * distances savings data / Best parameters: 3 - 4 - 3
                # 8 - Distances data * energies data * energies savings data / Best parameters: 3 - 4 - 3
                # 9 - Distances data * energies data * distances savings data * capacity utilization data / Best parameters: 2 - 4 - 4 - 2
                # 10 - Distances data * energies data * distances savings data * capacity utilization data / Best parameters: 2 - 4 - 4 - 2
                # 11 - Distances data * energies data * distances savings data * energies savings data * capacity utilization data / Best parameters: 2 - 4 - 3 - 4 - 1
                heuristic_type=2,
                # Importance of pheromones trace. Recommended alpha value in: [1, 4]
                alpha=3,
                # Importance of distance heuristic information. Recommended beta value in: [2, 5]
                beta=2,
                # Importance of energy heuristic information. Recommended gamma value in: [2, 5]
                gamma=3,
                # Importance of distances's Savings Algorithm information. Recommended delta value in: [0, 2]
                delta=1,
                # Importance of energies's Savings Algorithm information. Recommended eta value in: [0, 2]
                eta=1,
                # Importance of vehicle's capacity utilization. Recommended mi value in: [0, 2]
                mi=0,
                # Define the strategy used for pheromone updating. - options: | 0 - 1 |
                # The value 0 is for the classic function in ACO: 1 / Best_Solution_Quality
                # The value 1 if for the improve function know as Ant Weight Strategy.
                pheromone_updating_strategy=0,
                local_ant_update_pheromones=False,
                best_iteration_ant_update_pheromones=False,
                best_global_ant_update_pheromones=True,
                penalize_worst_solution=True,
                mutate_pheromones_matrix=True,
                # Evaporation rate of the pheromones by the next function: phoromone =  (1 - p) * pheromone. Recommended value for p: 0.02.
                p=0.02,
                # Mutation probability for every row of the pheromones matrix, such as: mutate if (random number between 0 and 1) <= Pm
                # Recommended value for Pm: 0.3.
                Pm=0.3,
                # Mutation intensity for the pheromones matrix. Recommended value for sigma in: [2, 4]
                sigma=4,
                # Constant used in Pseudorandom Transition Rule. This is a probability of choice the next node by argmax, define in Ant Colony System.
                # Recommended value for l0 in: [0.2, 0.4]
                l0=0.3,
                # Constant used for a lot of functions, such as: f(.) = H / specific_values.
                H=1,
                ls_ant_solution=False,
                ls_best_iteration=True,
                ls_best_global=False,
                use_normalized_matrix=False,
                print_instance=False,
                print_clusters=False,
                print_solution=False,
                print_distance_matrix=False,
                print_energy_matrix=False,
                print_distance_saving_matrix=False,
                print_energy_saving_matrix=False,
                print_combination_matrix=False,
                print_pheromone_matrix=False,
                output_sol_img=f'{BASE_RESULTS_FOLDER}/{instance}/imgs/{file_name}.png'
                )
            
            solution_energy, solution_distance, solution_time = bwacs.solve()
            solution_list.append((solution_energy, solution_distance, solution_time))
        except:
            instances_with_errors.append(instance)
            break
    
    print(f'Solutions for instance {instance}: {solution_list}')
    print(f'{RUNS} runs for instance {instance} finished at {datetime.now().strftime("%Y%m%d_%H%M%S")}')
    print(f'Best solution for instance {instance}: {min(solution_list, key=lambda x: x[0])}')
    print(f'Worst solution for instance {instance}: {max(solution_list, key=lambda x: x[0])}')

['CMT1', 'CMT2', 'CMT3', 'CMT4', 'CMT5', 'CMT6', 'CMT7', 'CMT8', 'CMT9', 'CMT10', 'CMT11', 'CMT12', 'CMT13', 'CMT14']


In [ ]:
import os
from datetime import datetime
from free_bwacs_model import FreeBWACS
%matplotlib inline

solutions_list = []
max_runs = 10
dct_with_header = False
save_results = True
_instance = 'CMT2'
_cluster_type = 'K-MEANS'

# Creación del archivo de texto plano con los resultados
now = datetime.now()
dt_string = now.strftime('%Y%m%d_%H-%M-%S')
aco_cluster = 'Free Ant + ' + _cluster_type  # Titulo del archivo .txt
# FA o RA (FREE ANT - RESTRICTED ANT) para el titulo del archivo
ant_type = 'FA'
# KMEN o KMED (K-MEANS - K-MEDOIDS) para el titulo del archivo
cluster_type = _cluster_type.replace('-', '').upper()[0:4]
instance = _instance
file_folder = 'instances_results/' + instance
file_folder_img = 'instances_results/' + instance + '/img_results'
if not os.path.exists(file_folder):
    os.makedirs(file_folder)
if not os.path.exists(file_folder_img):
    os.makedirs(file_folder_img)
file_name = ant_type + '_' + cluster_type + '_' + dt_string + '.txt'

for run in range(max_runs):
    aco_model = FreeBWACS(
        # Instance path + filename
        instance='CVRPLIB/CMT/' + _instance,
        # Max nodes selected
        max_nodes=9999,
        # Cluster type - options: | kmedoids - kmeans |
        cluster_type=_cluster_type.replace('-', '').lower(),
        # Metric distance - options: | euclidian - manhattan |
        metric='euclidian',
        # Tare for the heterogenoeus vehicles, this value is a percentage used by the next function: Vehicle Weight = Max Capacity * tare
        tare_percentage=0.15,
        # Max number of iterations
        max_iterations=100,
        # Define the total number of ants per iteration, through the next function: Total Ant = Total Nodes / total_ant_divider
        total_ant_divider=1,
        # Only accept solutions with the numbers of routes equal to K-Optimum - options: | True - False |
        only_k_optimum=True,
        # Place each ant in a different node of the candidate node list. Otherwise each ant will start its journey in the depot node.
        # options: | True - False |
        start_ant_on_best_nodes=True,
        # heuristic_type will let you choice what information gotta build the heuristic information matrix
        # 0 - Only distances data / Best beta: 5
        # 1 - Only energies data / Best gamma: 5
        # 2 - Distances data * energies data / Best parameters: 5 - 5
        # 3 - Only distances data of Savings algorithm / Best delta: 1
        # 4 - Only energies data of Savings algorithm / Best eta: 2
        # 5 - Distances data * distances savings data / Best parameters: 4 - 5
        # 6 - Energies data * energies savings data / Best parameters: 5 - 5
        # 7 - Distances data * energies data * distances savings data / Best parameters: 3 - 4 - 3
        # 8 - Distances data * energies data * energies savings data / Best parameters: 3 - 4 - 3
        # 9 - Distances data * energies data * distances savings data * capacity utilization data / Best parameters: 2 - 4 - 4 - 2
        # 10 - Distances data * energies data * distances savings data * capacity utilization data / Best parameters: 2 - 4 - 4 - 2
        # 11 - Distances data * energies data * distances savings data * energies savings data * capacity utilization data / Best parameters: 2 - 4 - 3 - 4 - 1
        heuristic_type=2,
        # Importance of pheromones trace. Recommended alpha value in: [1, 4]
        alpha=3,
        # Importance of distance heuristic information. Recommended beta value in: [2, 5]
        beta=2,
        # Importance of energy heuristic information. Recommended gamma value in: [2, 5]
        gamma=3,
        # Importance of distances's Savings Algorithm information. Recommended delta value in: [0, 2]
        delta=1,
        # Importance of energies's Savings Algorithm information. Recommended eta value in: [0, 2]
        eta=1,
        # Importance of vehicle's capacity utilization. Recommended mi value in: [0, 2]
        mi=0,
        # Define the strategy used for pheromone updating. - options: | 0 - 1 |
        # The value 0 is for the classic function in ACO: 1 / Best_Solution_Quality
        # The value 1 if for the improve function know as Ant Weight Strategy.
        pheromone_updating_strategy=0,
        # Determine if every single ant can update the pheromones whit - options: | True - False |
        local_ant_update_pheromones=False,
        # Determine if the best ant of current iteration can update the pheromones - options: | True - False |
        best_iteration_ant_update_pheromones=False,
        # Determine if the best global ant can update the pheromones - options: | True - False |
        best_global_ant_update_pheromones=True,
        # Defines if the worst solution of the current iteration should penalize the pheromone arcs. - options: | True - False |
        penalize_worst_solution=True,
        # Want to mutate the pheromones matrix? - options: | True - False |
        mutate_pheromones_matrix=True,
        # Evaporation rate of the pheromones by the next function: phoromone =  (1 - p) * pheromone. Recommended value for p: 0.02.
        p=0.02,
        # Mutation probability for every row of the pheromones matrix, such as: mutate if (random number between 0 and 1) <= Pm
        # Recommended value for Pm: 0.3.
        Pm=0.3,
        # Mutation intensity for the pheromones matrix. Recommended value for sigma in: [2, 4]
        sigma=4,
        # Constant used in Pseudorandom Transition Rule. This is a probability of choice the next node by argmax, define in Ant Colony System.
        # Recommended value for l0 in: [0.2, 0.4]
        l0=0.3,
        # Constant used for a lot of functions, such as: f(.) = H / specific_values.
        H=1,
        # Want to do a Local Search to the solution of every single ant? - options: | True - False |
        ls_ant_solution=False,
        # Want to do a Local Search to the best iteration solution? - options: | True - False |
        ls_best_iteration=True,
        # Want to do a Local Search to the best global solution? - options: | True - False |
        ls_best_global=False,
        # Want to use the Normalized Matrix? - options: | True - False |
        use_normalized_matrix=False,
        # Use Matplotlib to draw a 2-D graph with the nodes of the problem. - option: | True - False |
        print_instance=False,
        # Use Matplotlib to draw the solution of clustering process. - option: | True - False |
        print_clusters=False,
        # Use Matplotlib to draw the final solution of the problem. - option: | True - False |
        print_solution=False,
        # Show the distance matrix in a html table. - option: | True - False |
        print_distance_matrix=False,
        # Show the energy matrix in a html table. - option: | True - False |
        print_energy_matrix=False,
        # Show the distance savings matrix in a html table. - option: | True - False |
        print_distance_saving_matrix=False,
        # Show the energy savings matrix in a html table. - option: | True - False |
        print_energy_saving_matrix=False,
        # Show the final heuristic matrix in a html table. - option: | True - False |
        print_combination_matrix=False,
        # Show the pheromone matrix in a html table. - option: | True - False |
        print_pheromone_matrix=False,
        # Output folder/name solution draw
        output_sol_img=file_folder_img + '/' + \
        str(run + 1) + '__' + ant_type + '_' + \
        cluster_type + '_' + dt_string + '.png'
    )

    solution_energy, solution_distance, solution_time = aco_model.solve()
    solutions_list.append((solution_energy, solution_distance, solution_time))

print(solutions_list)
print("Best Solution: " + str(min(solutions_list, key=lambda x: x[0])) + " on run: " + str(solutions_list.index(min(solutions_list, key=lambda x: x[0])) + 1))

if (save_results):
    file = open(f'{file_folder}/{file_name}', 'a')
    if dct_with_header:
        file.write('Resultados para la ' + instance + ' (' +
                aco_cluster + ') [energia, distancia, tiempo de ejecución]\n\n')
    for i, solution in enumerate(solutions_list):
        if dct_with_header:
            file.write(str(i + 1) + '| ' + str(solution) + '\n')
        else:
            str_sol = str(solution).replace('(', '').replace(')',
                                                            '').replace(',', '').replace('.', ',')
            file.write(str_sol + '\n')

    file.close()
